#Ch. 4 Exercises
<ol>
<li>[Histogram Filter for 1-D Car](#Histogram_car)</li>
<li>[Histogram Filter for 2-D Robot](#Histogram_robot)</li>
<li>[Partical Filter Bias](#Partical_Filter_bais)</li>
<li>[Partical Filter 1-D Car](#Partical_Filter_car)</li>
<li>[Partical Filter 2-D Robot](#Partical_Filter_robot)</li>
</ol>

## 1. Histogram Filter for 1-D Car <a id="Histogram_car"></a>

(a) Implement a histogram filter for the car system from chapter 3.

State Vectors:
\begin{equation}
\mu = \left[\begin{array}{c}
x \\
\dot{x} \end{array}\right], \\
\space
\ddot{x}_{mean} = 0, \ddot{x}_{\sigma^{2}} = 1
\end{equation}


a robot at $bin_{j}$ at $t=-1$ will end up in $bin_{k}$ at $t=0$ if
\begin{equation}
x_{k} - x_{j} = \frac{1}{2}(\dot{x}_k + \dot{x}_j) \\
\ddot{x} = (\dot{x}_k - \dot{x}_j)
\end{equation}


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%pylab inline

#t=0

xs = np.linspace(-6,6,21)
vs = np.linspace(6,-6,21)

x_step = abs(xs[1]-xs[0])
v_step = abs(vs[1]-vs[0])

probs = np.zeros((len(xs), len(vs)))
probs[len(xs) / 2, len(vs) / 2] = 1

def _gaussian(mu, sigma_sq, x):
    """Compute the probablity of x of a gaussian centered at mu with variation sigma_sq"""
    return (2*np.pi*sigma_sq)**-0.5 *np.exp(-(x-mu)**2/(2*sigma_sq))
# Make it vectorized
gaussian = np.vectorize(_gaussian)
 

def hist_filt(probs_t_1, xs, vs):
    probs_bar_t = np.zeros(probs_t_1.shape)
    for i, x_t in enumerate(xs):
        for j, v_t in enumerate(vs):
            for k, x_t_1 in enumerate(xs):
                for l, v_t_1 in enumerate(vs):
                    # This is super hacky, but required to validate that the robot will actually be at
                    # the desired state. (There is probably a more elegant solution to this, but I don't know
                    # what it is...
                    if abs(x_t - x_t_1 - 0.5 * (v_t + v_t_1)) <= x_step: 
                        probs_bar_t[j,i] += probs_t_1[k,l] * gaussian(mu=0, sigma_sq=1, x=(v_t - v_t_1))
                        
    return probs_bar_t

# Plot the results
fig = plt.figure(figsize=(15,5))
for i in xrange(6):
    ax = fig.add_subplot(1,6,i+1, aspect='equal')
    ax.contour(xs, vs, probs)
    probs = hist_filt(probs, xs, vs)

plt.show()

In [ ]:
#(b) Measurement update
# measurement sigma^2 = 10

def hist_measurement_update(probs_bar_t, z_t, z_sigma):
    probs_t = np.zeros(probs_bar_t.shape)
    for i, x_t in enumerate(xs):
        for j, v_t in enumerate(vs):
            probs_t[j,i] = probs_bar_t[j,i] * gaussian(x_t, z_sigma, z_t)
    return probs_t
            
z_sigma = 10

probs = np.zeros((len(vs), len(xs)))
probs[len(vs) / 2, len(xs) / 2] = 1

for i in xrange(5):
    probs = hist_filt(probs, xs, vs)

probs_before = probs
probs_after = hist_measurement_update(probs_before, 5, np.sqrt(z_sigma))

fig = plt.figure(figsize=(9,9))
for i, p in enumerate((probs_before, probs_after)):
    ax = fig.add_subplot(1,2,i+1, aspect='equal')
    ax.contour(xs, vs, p)

plt.show()



#2. Histogram filter for 2D robot <a id="Histogram_robot"></a>

(a) Propose a state histogram filter for the robot in chapter 2.

The histogram is a 3d matrix with dimensions $\theta, y, x$

In [ ]:
xs = np.linspace(-1.5, 1.5, 11)
ys = np.linspace(1.5, -1.5, 11)
thetas = np.linspace(-np.pi, np.pi, 11, endpoint=False)

x_step = abs(xs[1]-xs[0])
y_step = abs(ys[1]-ys[0])
theta_step = abs(thetas[1]-thetas[0])

probs = np.zeros((len(thetas), len(ys), len(xs)))

x_mu = y_mu = 0
x_sigma = y_sigma = 0.1 # sigma^2 = 0.01

# Initial probabilities
for i, x in enumerate(xs):
    for j, y in enumerate(ys):
        probs[:,j,i] = pylab.normpdf(x, x_mu, x_sigma) * pylab.normpdf(y, y_mu, y_sigma)

probs = probs / probs.sum() # Normalize

fig = plt.figure(figsize=(17,5))
for i in xrange(len(thetas)):
    ax = fig.add_subplot(1,len(thetas),i+1, aspect='equal')
    ax.contour(xs, ys, probs[i,:,:])
    ax.set_title('Theta = %.1f' % thetas[i])

plt.show()

(b) Add the movement prediction

In [ ]:
#(b) Add the movement prediction

def hist_movement(probs_t_1, xs, ys, thetas, d):
    #Since we are moving with no noise, we can cheat a bit and work with each theta independently
    probs_bar_t = np.zeros(probs_t_1.shape)
    for k, t in enumerate(thetas):
        delta_x, delta_y = np.cos(t)*d, np.sin(t)*d
        delta_x_idx, delta_y_idx = delta_x // x_step, delta_y // y_step        
        for i, x_t in enumerate(xs):
            for j, y_t in enumerate(ys):
                if (i + delta_x_idx) > 0 and (i + delta_x_idx) < len(xs) and \
                   (j + delta_y_idx) > 0 and (j + delta_y_idx) < len(ys):
                    probs_bar_t[k, j, i] = probs_t_1[k, j + delta_y_idx, i + delta_x_idx]
                else:        
                    probs_bar_t[k,j,i] = 0
    return probs_bar_t

                        
probs_move = hist_movement(probs, xs, ys, thetas, 1)

fig = plt.figure(figsize=(17,5))
for i in xrange(len(thetas)):
    ax = fig.add_subplot(1,len(thetas),i+1, aspect='equal')
    ax.contour(xs, ys, probs_move[i,:,:])
    ax.set_title('Theta = %.1f' % thetas[i])

plt.show()

(c) Measurement

We measure the $x$ coordinate of the robots with covariance $Q = 0.01$ ($\sigma = 0.1$)  
The position of the robot isn't specified in the text, so I'm going to assume $z = 0.0$

In [ ]:
def hist_measurement_update_2(probs_bar_t, z_t, z_sigma):
    probs_t = np.zeros(probs_bar_t.shape)
    for i, t_t in enumerate(thetas):
        for j, x_t in enumerate(xs):
            for k, y_t in enumerate(ys):
                probs_t[i,k,j] = probs_bar_t[i,k,j] * pylab.normpdf(x_t, z_t, z_sigma)
    
    return probs_t / probs_t.sum() # Normalize the result

probs_measure = hist_measurement_update_2(probs_move, 0.0, 0.1)

fig = plt.figure(figsize=(17,5))
for i in xrange(len(thetas)):
    ax = fig.add_subplot(1,len(thetas),i+1, aspect='equal')
    ax.contour(xs, ys, probs_measure[i,:,:], np.linspace(0,probs_measure.max(),10))
    ax.set_title('Theta = %.1f' % thetas[i])

plt.show()

#3. Partical Filter Bias <a id="Partical_Filter_bais"></a>

What happens if the number of particles is M=2? Give an example of were this leads to bias.

If we only use two particles, the measuremnt will affect the output, (unlike the M=1 case), but it's very likely that neither particle will be a good representation of the true robot pose, (particle deprivation). Also, it would be very suceptable to resampling only 1 particle and removing the particle variance.

#4. Partical Filter 1-D Car <a id="Partical_Filter_car"></a>

Implement a partical filter for the system in problem 1

In [ ]:
from collections import namedtuple
from random import gauss
from random import normalvariate

CarParticle = namedtuple('CarParticle', ['x', 'v'])

M = 100 # Number of particles
particles = []
for i in xrange(M):
    particles.append(CarParticle(0,0))


def car_particle_prediction_update(particles):
    new_particles = []
    for p in particles:
        a = normalvariate(mu=0, sigma=1)
        new_particles.append(CarParticle(p.x + p.v + 0.5*a, p.v + a))
    return new_particles
        

# Plot the results
fig = plt.figure(figsize=(15,5))
for i in xrange(6):
    xs, vs = [], []
    for p in particles:
        xs.append(p.x)
        vs.append(p.v)
    ax = fig.add_subplot(1,6,i+1, aspect='equal')
    ax.plot(xs, vs, 'kx')
    particles = car_particle_prediction_update(particles)   
    ax.set_xlim(-6, 6)
    ax.set_ylim(-6, 6)

plt.show()

In [ ]:
# now incorperate the measurement with sigma^2 = 10
from random import random

sigma = np.sqrt(10)

def low_variance_sampling(particles, weights):
    new_particles = []
    M = len(particles)
    M_inv = M**-1
    i = 0 # weight index
    r = 0  # initial weight offset
    while r == 0: # Force r != 0
        r = random() * M_inv
    
    c = weights[0]
    for m in xrange(M):
        U = r + m*M_inv
        while U > c:
            i = i + 1
            c = c + weights[i]
        new_particles.append(particles[i])

    return new_particles
    


def car_particle_measurement_update(particles, z_t, z_sigma):
    weights = []
    for p in particles:
        weights.append(pylab.normpdf(p.x, z_t, z_sigma))
    weights = np.array(weights)
    weights = weights / weights.sum() # Normalize
    
    new_particles = low_variance_sampling(particles, weights)
    return new_particles



M = 100 # Number of particles
particles = []
for i in xrange(M):
    particles.append(CarParticle(0,0))
    
for i in xrange(5):
    particles = car_particle_prediction_update(particles)
    
particles_before = particles

particles_after = car_particle_measurement_update(particles, 5, sigma)
    
# Plot the results
fig = plt.figure(figsize=(9,9))
for i, pp in enumerate((particles_before, particles_after)):
    ax = fig.add_subplot(1,2,i+1, aspect='equal')
    xs, vs = [], []
    for p in pp:
        xs.append(p.x)
        vs.append(p.v)
        ax.plot(xs, vs, 'kx')
        ax.set_xlim(-12, 12)
        ax.set_ylim(-12, 12)

plt.show()

#5. Partical Filter 2-D Robot <a id="Partical_Filter_robot"></a>

Implement a partical filter for the system in problem 1

In [ ]:
RobotParticle = namedtuple('RobotParticle', ['x', 'y', 'theta'])

x_sigma = 0.1
y_sigma = 0.1
theta_sigma = 100

M = 100 # Number of particles
particles = []
for i in xrange(M):
    x = normalvariate(mu=0, sigma=x_sigma)
    y = normalvariate(mu=0, sigma=y_sigma)
    theta = normalvariate(mu=0, sigma=theta_sigma)
    particles.append(RobotParticle(x,y,theta))

def robot_particle_prediction_update(particles, u):
    new_particles = []
    for p in particles:
        x = p.x + cos(p.theta)*u
        y = p.y + sin(p.theta)*u
        new_particles.append(RobotParticle(x, y, p.theta))
    return new_particles

particles_bar = robot_particle_prediction_update(particles, 1)

# Plot the results
fig = plt.figure(figsize=(9,9))
stick_len = 0.1
ax = fig.add_subplot(1,1,1, aspect='equal')
xs, ys, ts = [], [], []
for p in particles_bar:
    ax.plot(p.x, p.y, 'ko')
    ax.plot([p.x, p.x + stick_len*cos(p.theta)], [p.y, p.y + stick_len*sin(p.theta)], 'k')
ax.set_xlim(-1.5, 1.5)
ax.set_ylim(-1.5, 1.5)
plt.show()

In [ ]:
# Add the measurement

def robot_particle_measurement_update(particles, z_t, z_sigma):
    weights = []
    for p in particles:
        weights.append(pylab.normpdf(p.x, z_t, z_sigma))
    weights = np.array(weights)
    weights = weights / weights.sum() # Normalize
    
    new_particles = low_variance_sampling(particles, weights)
    return new_particles


particles_after = robot_particle_measurement_update(particles_bar, 0, .1)

# Plot the results
fig = plt.figure(figsize=(9,9))
stick_len = 0.1
ax = fig.add_subplot(1,1,1, aspect='equal')
xs, ys, ts = [], [], []
for p in particles_after:
    ax.plot(p.x, p.y, 'ko')
    ax.plot([p.x, p.x + stick_len*cos(p.theta)], [p.y, p.y + stick_len*sin(p.theta)], 'k')

ax.set_xlim(-1.5, 1.5)
ax.set_ylim(-1.5, 1.5)
plt.show()